In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from osgeo import gdal, osr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as matplotlib

CA_seas_slopes = sio.loadmat('CA_seas_slopes.mat')  
slope_all = np.squeeze(np.array(CA_seas_slopes['slope_all']))

CA_keras_slopes_mse = sio.loadmat('CA_keras_slopes_adam_mse_historical_MB.mat')  
llon_1 = np.squeeze(np.array(CA_keras_slopes_mse['llon_1']))
llon_2 = np.squeeze(np.array(CA_keras_slopes_mse['llon_2']))
orient = np.squeeze(np.array(CA_keras_slopes_mse['orient']))
rmse = np.squeeze(np.array(CA_keras_slopes_mse['rmse']))
rmse_weighted = np.squeeze(np.array(CA_keras_slopes_mse['rmse_weighted']))
bias = np.squeeze(np.array(CA_keras_slopes_mse['bias']))

CA_keras_slopes_mse = sio.loadmat('CA_keras_seas_slopes.mat')  
slope = np.squeeze(np.array(CA_keras_slopes_mse['slope_CA_MSL_MHHW']))
# slope[np.isnan(tide_amp)] = np.NaN

CA_keras_beach_width = sio.loadmat('CA_keras_beach_width.mat')  
rmse_field = np.squeeze(np.array(CA_keras_beach_width['rmse_field']))
shore_std = np.squeeze(np.array(CA_keras_beach_width['shore_std']))


In [ ]:
plt.close('all')

slope = np.array(slope)
# %matplotlib widget
fig, (ax0,ax1,ax2,ax3,ax4) = plt.subplots(nrows=1,ncols=5,sharex=False)
plt.subplots_adjust(wspace=0.2)
plt.subplots_adjust(hspace=1)
fig.set_size_inches(16, 2)

ax0 = sns.histplot(bias,stat='percent',ax=ax0)
ax0.set_xlim([-10,10])
ax0.set(xlabel='Bias for Test Data\n(m)')
ax0.text(-9.75,5.9,'(f)',fontsize=12,color='k')

ax1 = sns.histplot(rmse,stat='percent',ax=ax1)
ax1.set_xlim([0,23])
ax1.set_xlabel('RMSE\nfor Test Data (m)')
ax1.text(0.4,7.1,'(g)',fontsize=12,color='k')
ax1.set_ylabel('')

ax2 = sns.histplot(rmse_weighted,stat='percent',ax=ax2)
ax2.set_xlim([0,1.5])
ax2.axvline(x=1.0,color='black',alpha=0.5)
ax2.set_xlabel('Normalized RMSE\nfor Test Data')
ax2.text(0.02,6.61,'(h)',fontsize=12,color='k')
frac_ = np.sum(rmse_weighted>1.0)/np.sum(rmse_weighted>0)
ax2.text(1.05,4.25,str(np.round(frac_*100*10)/10) + '%\nvalues\n> 1.0')
ax2.set_ylabel('')

ax3 = sns.histplot(rmse_field,stat='percent',ax=ax3)
ax3.set_xlim([0,23])
ax3.set_xlabel('RMSE\n for Surveys (m)')
ax3.text(0.4,10.75,'(i)',fontsize=12,color='k')
ax3.set_ylabel('')

rmse_weighted_field = rmse_field/shore_std
ax4 = sns.histplot(rmse_weighted_field,stat='percent',ax=ax4)
ax4.set_xlim([0,1.75])
ax4.axvline(x=1.0,color='black',alpha=0.5)
ax4.set_xlabel('Normalized RMSE\nfor Surveys')
ax4.text(0.02,11,'(j)',fontsize=12,color='k')
frac_ = np.sum(rmse_weighted_field>1.0)/np.sum(rmse_weighted_field>0)
ax4.text(1.05,7,str(np.round(frac_*100*10)/10) + '%\nvalues\n>1.0')
ax4.set_ylabel('')
# ax2 = sns.histplot(slope_all,bins=40,stat='percent',ax=ax2,color=sns.color_palette("Paired")[5])
# sns.histplot(slope,bins=40,stat='percent',ax=ax2)
# ax2.text(0.085,9,'Beach Slope from \nVos et al., (2020)',fontsize=12,color=sns.color_palette("Paired")[5])
# ax2.text(0.085,6,'Beach Slope from \nthis study',fontsize=12,color=sns.color_palette("Paired")[1])
# ax2.text(0.005,10.5,'(f)',fontsize=12,color='k')
# ax2.set_xlabel('Beach Slope')
# ax2.set_xlim([0,0.25])

# sns.histplot(slope,stat='percent',bins=40)
plt.savefig('figures/histograms.pdf',bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from osgeo import gdal, osr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.io as sio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as matplotlib
import colorcet as cc
from matplotlib.colors import LinearSegmentedColormap

CA_seas_slopes = sio.loadmat('CA_seas_slopes.mat')  
slope_all = np.squeeze(np.array(CA_seas_slopes['slope_all']))
shore_winterspring = np.squeeze(CA_seas_slopes['shore_winterspring'])
shore_summerfall = np.squeeze(CA_seas_slopes['shore_summerfall'])
shore_95_winterspring = np.squeeze(CA_seas_slopes['shore_95_winterspring'])
shore_95_summerfall = np.squeeze(CA_seas_slopes['shore_95_summerfall'])
storm_diff = np.squeeze(np.array(CA_seas_slopes['storm_diff']))

CA_keras_slopes_mse = sio.loadmat('CA_keras_slopes_adam_mse.mat')  
llon_1 = np.squeeze(np.array(CA_keras_slopes_mse['llon_1']))
llon_2 = np.squeeze(np.array(CA_keras_slopes_mse['llon_2']))
orient = np.squeeze(np.array(CA_keras_slopes_mse['orient']))
seasonal_wave = np.squeeze(np.array(CA_keras_slopes_mse['seasonal_wave']))
interann_wave = np.squeeze(np.array(CA_keras_slopes_mse['interann_wave']))
tide_amp = np.squeeze(np.array(CA_keras_slopes_mse['tide_amp']))
residual_std = np.squeeze(np.array(CA_keras_slopes_mse['residual_std']))
max_seas_doy = np.squeeze(np.array(CA_keras_slopes_mse['max_seas_doy']))
peak_time = np.squeeze(np.array(CA_keras_slopes_mse['peak_time']))
lons = np.array(llon_1)[:,0]
lats = np.array(llon_1)[:,1]

CA_keras_slopes_mse = sio.loadmat('CA_keras_slopes_adam_mse_historical_MB.mat')  
llon_1 = np.squeeze(np.array(CA_keras_slopes_mse['llon_1']))
llon_2 = np.squeeze(np.array(CA_keras_slopes_mse['llon_2']))
orient = np.squeeze(np.array(CA_keras_slopes_mse['orient']))
rmse = np.squeeze(np.array(CA_keras_slopes_mse['rmse']))
rmse_weighted = np.squeeze(np.array(CA_keras_slopes_mse['rmse_weighted']))
bias = np.squeeze(np.array(CA_keras_slopes_mse['bias']))


In [ ]:
CA_keras_slopes_mse = sio.loadmat('CA_keras_seas_slopes.mat')  
slope = np.squeeze(np.array(CA_keras_slopes_mse['slope_CA_MSL_MHHW']))
slope[np.isnan(tide_amp)] = np.NaN

CA_keras_beach_width = sio.loadmat('CA_keras_beach_width.mat')  
mse_field = np.squeeze(np.array(CA_keras_beach_width['mse_field']))

idx = np.squeeze(np.where(np.squeeze(np.logical_or(np.array(slope)>0.2,np.array(slope)<0))))
mse[idx] = np.NaN
bias[idx] = np.NaN
slope[idx] = np.NaN

# pdata[pdata<0.2] = np.NaN
fig, (ax0,ax1,ax2) = plt.subplots(nrows=1,ncols=3,figsize=(12,8))
plt.subplots_adjust(wspace=0.55)
plt.subplots_adjust(hspace=0.0)

pdata = np.array(slope)
plot_along_ca_beach(ax0,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Slope For\nAll Beaches',ylim=[0,0.18],yticks=True,nosmooth=False)
ax0.set_ylabel('Along-Coast Distance (km)',fontsize=12)

ax0.text(-0.01,along_dist_names[0],'Torrey\nPines', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[1],'Newport\nBeach', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[2],'Santa\nMonica', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[3],'Santa\nBarbara', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[4],'Ragged\nPoint', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[5],'Monterey', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[6],'Ocean\nBeach', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[7],'Bodega\nBay', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[8],'Mendocino', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[9],'King\nRange', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[10],'Eureka', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.text(-0.01,along_dist_names[11],'Crescent\nCity', horizontalalignment='right', verticalalignment='center', fontsize=12)
ax0.yaxis.tick_right()
ax0.yaxis.set_label_position('right')
ax0.text(0.13,1635,'(a)',fontsize=12)  
ax0.grid(False)
ax0.set_xlim([0,0.15])

from matplotlib.patches import Rectangle
rect = Rectangle((0, 0), 50, 150, linewidth=2, edgecolor=sns.color_palette("Paired")[9], facecolor='none')

# Now, let's add this rectangle to the subplot.
ax0.add_patch(rect)
ax0.spines['bottom'].set_color(sns.color_palette("Paired")[9])

pdata = slope
plot_along_ca_beach(ax1,along_dist_transect,closest_distance,pdata,along_dist_names_socal,ylabel='Slope For Select\nSouthern California\nBeaches',ylim=[0,0.2],yticks=True,nosmooth=False,legend=True,cbar_label='DNN Beach\nSlope')
ax1.set_ylim([50,95])

ax1.text(0.152,along_dist_names_socal[3],'SIO Pier', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[4],'Torrey\nPines', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[5],'San\nDieguito', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[6],'San\nElijo', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[7],'Moonlight', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[8],'Batiquitos', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax1.text(0.152,along_dist_names_socal[9],'Tamarack', horizontalalignment='left', verticalalignment='center', fontsize=12)
# ax1.text(0.21,along_dist_names_socal[10],'Oceanside\nPier', horizontalalignment='left', verticalalignment='center', fontsize=12)
# ax1.text(0.21,along_dist_names_socal[11],'San Onofre', horizontalalignment='left', verticalalignment='center', fontsize=12)

ax1.spines['top'].set_color(sns.color_palette("Paired")[9])
ax1.spines['bottom'].set_color(sns.color_palette("Paired")[9])
ax1.spines['left'].set_color(sns.color_palette("Paired")[9])
ax1.spines['right'].set_color(sns.color_palette("Paired")[9])

pdata = slope_all
pdata[closest_distance > 0.8] = np.nan
pdata[np.gradient(along_dist_transect) > 10] = np.nan
smoothed_pdata = smooth_data(along_dist_transect, pdata, 0.25)
smoothed_pdata[closest_distance > 0.8] = np.nan
smoothed_pdata[np.gradient(along_dist_transect) > 10] = np.nan
ax1.plot(pdata, along_dist_transect, color=sns.color_palette("Paired")[1],label='Vos et al.,\n(2020)')

CA_MOP_slopes = sio.loadmat('seasonal_slope/CA_MOP_slopes.mat')  
mop_slope = np.squeeze(np.array(CA_MOP_slopes['mop_slope']))
mop_bmax = np.squeeze(np.array(CA_MOP_slopes['mop_bmax']))*2.
coastsat_slope_seasonal = np.squeeze(np.array(CA_MOP_slopes['coastsat_slope_seasonal']))*2.

pdata = mop_slope
ax1.plot(pdata, along_dist_transect, color=sns.color_palette("Paired")[3],label='Surveys')
ax1.legend(loc='upper right', bbox_to_anchor=(1, 0.2))
ax1.grid(False)
ax1.set_xlim([0,0.15])
# rect = Rectangle((0, 0), 50, 150, linewidth=2, edgecolor=sns.color_palette("Paired")[9], facecolor='none')
# ax1.add_patch(rect)
ax1.text(0.13,94,'(b)',fontsize=12)  

pdata = -1*coastsat_slope_seasonal
plot_along_ca_beach(ax2,along_dist_transect,closest_distance,pdata,along_dist_names_socal,ylabel='Seasonal Change in\nBeach Slope',ylim=[0,0.2],yticks=True,nosmooth=False,legend=True,cbar_label='DNN\nBeach\nSlope\nChange\n(Feb\nminus\nSep)')
ax2.set_ylim([50,95])

ax2.text(0.204,along_dist_names_socal[3],'SIO Pier', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[4],'Torrey\nPines', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[5],'San\nDieguito', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[6],'San\nElijo', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[7],'Moonlight', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[8],'Batiquitos', horizontalalignment='left', verticalalignment='center', fontsize=12)
ax2.text(0.204,along_dist_names_socal[9],'Tamarack', horizontalalignment='left', verticalalignment='center', fontsize=12)

ax2.spines['top'].set_color(sns.color_palette("Paired")[9])
ax2.spines['bottom'].set_color(sns.color_palette("Paired")[9])
ax2.spines['left'].set_color(sns.color_palette("Paired")[9])
ax2.spines['right'].set_color(sns.color_palette("Paired")[9])

pdata = -1*mop_bmax
ax2.plot(pdata, along_dist_transect, color=sns.color_palette("Paired")[3],label='Surveys')
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.3))
ax2.grid(False)
ax2.set_xlim([-0.1,0.2])
ax2.text(0.0875,94,'(c)',fontsize=12)  

plt.savefig('figures/beach_slope_profiles.pdf',bbox_inches='tight')
plt.savefig('figures/beach_slope_profiles.png',bbox_inches='tight')